In [0]:
-- ============================================================
-- CAMADA: BRONZE
-- ENTIDADE: Clientes
--
-- OBJETIVO:
-- Criar a view bronze de clientes a partir da camada RAW,
-- aplicando apenas padronizações técnicas (sem regra de negócio).
--
-- FONTE:
-- workspace.default.cliente
--
-- DESTINO:
-- workspace.olist_bronze.clientes_bronze
-- ============================================================
CREATE SCHEMA IF NOT EXISTS workspace.olist_bronze;

CREATE OR REPLACE VIEW workspace.olist_bronze.clientes_bronze AS
SELECT
    CASE
        WHEN customer_id RLIKE '^[a-zA-Z0-9_-]+$'
        THEN customer_id
        ELSE NULL
    END AS customer_id,

    CASE
        WHEN customer_unique_id RLIKE '^[a-zA-Z0-9_-]+$'
        THEN customer_unique_id
        ELSE NULL
    END AS customer_unique_id,

    CAST(customer_zip_code_prefix AS INT) AS customer_zip_code_prefix,

    TRANSLATE(
        LOWER(TRIM(customer_city)),
            'áàãâäéèêëíìîïóòõôöúùûüç',
            'aaaaaeeeeiiiiooooouuuuc'
            )           
                                          AS customer_city,

    UPPER(TRIM(customer_state))           AS customer_state

FROM workspace.default.cliente;